# Property calculation with cubic EoS

First it is necessary to import the ``component`` class and the equation of state (EoS). For these example we will use Peng-Robinson EoS (``preos``).

In [1]:
import numpy as np
from phasepy import component, preos

---
### Pure Fluids

A fluid is created (``water``) and then the EoS object is created. The ``eosp`` object includes the methods to evaluate properties from the equation of state, such as, densities, pressure, fugacity coefficients, chemical potential and some thermal derived properties (residual entropy, residual enthalpy, residual heat capacities and speed of sound).

**warning:** thermal derived properties are computed with numerical derivatives using $O(h^4)$ approximation.

In [2]:
water = component(name = 'water', Tc = 647.13, Pc = 220.55, Zc = 0.229, Vc = 55.948, w = 0.344861,
                GC = {'H2O':1}, Mw = 18.04)

eosp = preos(water)

The density of the fluid is computed with the ``eosp.density`` method. It requires the temperature, pressure and the aggregation state.

In [3]:
T = 340.  # K
P = 1.  # bar
# computed densities in mol/cm3
eosp.density(T, P, 'L'), eosp.density(T, P, 'V')

(0.045697369422639834, 3.5769380018112745e-05)

Similarly the pressure of the fluid can be computed at given molar volume and temperature using the ``eosp.pressure`` method. 

In [4]:
rhol = 0.045697369422639834
vl = 1./rhol

rhov = 3.5769380018112745e-05
vv = 1./rhov
eosp.pressure(T, vl), eosp.pressure(T, vv)

(array([1.]), array([1.]))

For pure fluids, the ``eos.psat`` method allows to compute the saturation pressure at given temperature. It returns the equilibrium pressure and molar volumes of the liquid and vapor phase. Similarly, the ``eos.tsat`` method allows to compute the saturation temperarute at given pressure.

The phase equilibria can be verified through fugacity coefficients using the ``eos.logfug`` method or by using chemical potentials with the ``eos.muad`` method. The chemical potentials require that dimensionless density and temperature.

In [5]:
Psat, vlsat, vvsat = eosp.psat(T)

# checking fugacity coefficients
logfugl, vlsat = eosp.logfug(T, Psat, 'L')
logfugv, vvsat = eosp.logfug(T, Psat, 'V')


# checking chemical potentials
Tfactor, Pfactor, rofactor, tenfactor, zfactor = eosp.sgt_adim(T)

Tad = T*Tfactor
rholad = 1/vlsat * rofactor
rhovad = 1/vvsat * rofactor

mul = eosp.muad(rholad, Tad)
muv = eosp.muad(rhovad, Tad)

print('Fugacity coefficients:', np.allclose(logfugl, logfugv ))
print('Chemical potential:', np.allclose(mul, muv ))

Fugacity coefficients: True
Chemical potential: True


C:\Users\Gusta\anaconda3\lib\site-packages\phasepy-0.0.50-py3.8-win-amd64.egg\phasepy\cubic\cubicpure.py:426: RuntimeWarning: divide by zero encountered in true_divide
  zfactor = np.sqrt(a/ci*10**5/100**6)*10**-10


In [6]:
# computing saturation temperature
P = 1.01325 # bar
Tsat, vlsat2, vvsat2 = eosp.tsat(P, T0=350.)

# checking fugacity coefficients
logfugl, vlsat = eosp.logfug(Tsat, P, 'L')
logfugv, vvsat = eosp.logfug(Tsat, P, 'V')


# checking chemical potentials
Tfactor, Pfactor, rofactor, tenfactor, zfactor = eosp.sgt_adim(Tsat)

Tad2 = Tsat*Tfactor
rholad2 = 1/vlsat2 * rofactor
rhovad2 = 1/vvsat2 * rofactor

mul = eosp.muad(rholad2, Tad2)
muv = eosp.muad(rhovad2, Tad2)

print('Fugacity coefficients:', np.allclose(logfugl, logfugv ))
print('Chemical potential:', np.allclose(mul, muv ))

Fugacity coefficients: True
Chemical potential: True


The ``eosp`` object also includes the calculation of some thermal derived properties such as residual entropy (``eosp.EntropyR``), residual enthalpy (``eosp.EnthalpyR``), residual isochoric heat capacity (``eosp.CvR``), , residual isobaric heat capacity (``eosp.CpR``).

For the speed of sound calculation (``eosp.speed_sound``) the ideal gas heat capacities are required, in the example the isochoric and isobaric ideal gas contribution are set to $3R/2$ and $5R/2$, respectively. Better values of ideal gas heat capacities contribution can be found from empirical correlations, such as the provided by DIPPR 801.

In [7]:
# vaporization entropy in J/mol K
Svap = eosp.EntropyR(T, Psat, 'V') - eosp.EntropyR(T, Psat, 'L')

# vaporization enthalpy in J/mol
Hvap = eosp.EnthalpyR(T, Psat, 'V') - eosp.EnthalpyR(T, Psat, 'L')

# isochoric and isobaric residual heats capacities in J / mol K
cvr = eosp.CvR(T, P, 'L')
cpr = eosp.CpR(T, P, 'L')

# ideal gas heat capacities, better values can be obtained with DIPPR 801 correlations
r = 8.314  # J / mol K
CvId = 3*r/2
CpId = 5*r/2
w = eosp.speed_sound(T, P, 'V', CvId=CvId, CpId=CpId)

print('Vaporization Entropy : ', Svap, 'J / mol K')
print('Vaporization Enthalpy : ', Hvap, 'J / mol')
print('Residual isochoric heat capacity : ', cvr, 'J / mol K')
print('Residual isobaric heat capacity : ', cpr, 'J / mol K')
print('Speed of sound : ', w, 'm / s')

Vaporization Entropy :  [128.64101949] J / mol K
Vaporization Enthalpy :  [43737.94662499] J / mol
Residual isochoric heat capacity :  [30.75238706] J / mol K
Residual isobaric heat capacity :  [47.74391561] J / mol K
Speed of sound :  [508.68253869] m / s


---
### Fluid mixtures

The EoS can be used for mixtures. The mixture is first created from two pure fluids using the ``mixture`` class. 
In this example the mixture is modeled with the Peng-Robinson EoS using the MHV mixing rule and the UNIFAC activity coefficient model. 

The ``eos`` object includes the methods to evaluate properties from the equation of state, such as, densities, pressure, fugacity coefficients, chemical potential and some thermal derived properties (residual entropy, residual enthalpy, residual heat capacities and speed of sound).

**again:** thermal derived properties are computed with numerical derivatives using $O(h^4)$ approximation.

In [8]:
from phasepy import mixture

water = component(name = 'water', Tc = 647.13, Pc = 220.55, Zc = 0.229, Vc = 55.948, w = 0.344861,
                GC = {'H2O':1}, Mw = 18.04)


ethanol = component(name = 'ethanol', Tc = 514.0, Pc = 61.37, Zc = 0.241, Vc = 168.0, w = 0.643558,
                GC = {'CH3':1, 'CH2':1,'OH(P)':1}, Mw = 46.07)

mix = mixture(water, ethanol)

# or

mix = water + ethanol

mix.unifac()
eos = preos(mix, 'mhv_unifac')

The density of the fluid is computed with the ``eos.density`` method. It requires the composition, temperature, pressure and the aggregation state.

In [9]:
T = 340.  # K
P = 1.  # bar
x = np.array([0.3, 0.7])
eos.density(x, T, P, 'L'), eos.density(x, T, P, 'V')

(0.01905921137858399, 3.611599060036928e-05)

The pressure of the fluid mixture can be computed at given composition, molar volume and temperature using the ``eos.pressure`` method. 

In [10]:
rhol = eos.density(x, T, P, 'L')
vl = 1. / rhol

rhov = eos.density(x, T, P, 'V')
vv = 1. / rhov

eos.pressure(x, vl, T), eos.pressure(x, vv, T)

(0.999999999998181, 0.9999999999999996)

The effective fugacity coefficients can be computed at given composition, temperature, pressure and aggregation state using the ``eos.logfugef`` method. This functions returns the natural logarithm of the fugacity coefficients and the computed volume root.

The fugacity of the mixture can be computed with the ``eos.logfugmix`` method. It works similarly as the previous method. 

The chemical potential are computed at given dimensionless density vector ($\rho_i = x_i \rho b_0$) and temperature.


In [11]:
lnphi, v = eos.logfugef(x, T, P, 'L')
lnphimix, v = eos.logfugmix(x, T, P, 'L')

print('Partial molar property : ', np.allclose(lnphimix, np.dot(lnphi, x)))

rhofactor = eos.b[0]
rhoad = 1/v * rofactor
rhovector = x*rhoad
eos.muad(rhovector, T)


Partial molar property :  True


array([-0.05741901,  0.01584424])

The ``eos`` object also includes the calculation of some thermal derived properties such as residual entropy (``eos.EntropyR``), residual enthalpy (``eos.EnthalpyR``), residual isochoric heat capacity (``eos.CvR``), residual isobaric heat capacity (``eos.CpR``).

For the speed of sound calculation (``eos.speed_sound``) the ideal gas heat capacities are required, in the example, the isochoric and isobaric ideal gas contributions are set to $3R/2$ and $5R/2$, respectively. Better values of ideal gas heat capacities contribution can be found from empirical correlations, such as the provided by DIPPR 801.

In [12]:
# Thermal derived properties

Sr = eos.EntropyR(x, T, P, 'L')
Hr = eos.EnthalpyR(x, T, P, 'L')
Cvr = eos.CvR(x, T, P, 'L')
Cpr = eos.CpR(x, T, P, 'L')

# ideal gas heat capacities, better values can be obtained with DIPPR 801 correlations
r = 8.314  # J / mol K
CvId = 3*r/2
CpId = 5*r/2
w = eos.speed_sound(x, T, P, 'V', CvId=CvId, CpId=CpId)

print('Residual Entropy : ', Sr, 'J / mol K')
print('Residual Enthalpy : ', Hr, 'J / mol')
print('Residual isochoric heat capacity : ', Cvr, 'J / mol K')
print('Residual isobaric heat capacity : ', Cpr, 'J / mol K')
print('Speed of sound : ', w, 'm / s')

Residual Entropy :  -116.82399490452899 J / mol K
Residual Enthalpy :  -41118.71891716509 J / mol
Residual isochoric heat capacity :  49.63465165531006 J / mol K
Residual isobaric heat capacity :  80.144782348063 J / mol K
Speed of sound :  351.21516609921684 m / s


---
For further information about each method check out the documentation running: ``eos.function?``